In [ ]:
! nvidia-smi

Thu May  4 02:14:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pandas as pd
import os
import shutil
from shutil import copyfile
import tensorflow as tf
import keras
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Flatten
from tensorflow.keras.models import Model
from glob import glob
import os
import numpy as np

In [ ]:
import pandas as pd
import os
import shutil

# Dump all images into a folder and specify the path:
# data_dir = os.getcwd() + "/content/drive/MyDrive/all_data"

# Path to destination directory where we want subfolders
# dest_dir = os.getcwd() + "/content/drive/MyDrive/reorganised_data/"

# Read the csv file containing image names and corresponding labels
skin_df2 = pd.read_csv('/content/drive/MyDrive/HAM10000_metadata.csv')
skin_df2.head()


,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [ ]:
import os
import glob
train_path = '/content/drive/MyDrive/reorganised_data/Train'
test_path = '/content/drive/MyDrive/reorganised_data/Test'


# Get the list of class directories
class_dirs = [d for d in os.listdir(train_path) if os.path.isdir(os.path.join(train_path, d))]

# Iterate over the class directories and count the number of images
for class_dir in class_dirs:
    class_path = os.path.join(train_path, class_dir)
    num_images = len(glob.glob(os.path.join(class_path, '*.jpg')))  # Change the file extension to match your dataset
    print(f'{class_dir}: {num_images} images')

bkl: 299 images
nv: 275 images
df: 110 images
mel: 293 images
vasc: 122 images
bcc: 414 images
akiec: 257 images


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
from PIL import Image

In [ ]:
# import pandas as pd
# import os
# from shutil import copyfile

# # Load metadata CSV file
# metadata = pd.read_csv("C:/Users/choug/Downloads/archive (4)/HAM10000_metadata.csv")


# # Set the source directory path
# source_dir = 'C:/Users/choug/Downloads/archive (4)/all_data/'

# # Set the destination directory path
# dest_dir = 'C:/Users/choug/Downloads/archive (4)/data_reorganised/'

# # Create a list of unique class labels
# labels = metadata['dx'].unique().tolist()

# # Create a folder for each class label
# for label in labels:
#     os.makedirs(dest_dir + label + '/', exist_ok=True)

# # Copy images to new folders
# for i in range(len(metadata)):
#     file_name = metadata.iloc[i]['image_id'] + '.jpg'
#     label = metadata.iloc[i]['dx']
#     src = source_dir + file_name
#     dst = dest_dir + label + '/' + file_name
#     copyfile(src, dst)

# print("Images separated into class folders successfully!")

In [ ]:
import random

In [ ]:
# src_dir = '/content/drive/MyDrive/reorganised_data/Test/nv/'
# dst_dir = '/content/drive/MyDrive/extra_data/nv/'

# # Get a list of all the files in the source directory
# all_files = os.listdir(src_dir)

# # Randomly select 1000 files from the list
# files_to_move = random.sample(all_files, k=1225)

# # Iterate through the filenames and move the corresponding files to the destination directory
# for filename in files_to_move:
#     src_path = os.path.join(src_dir, filename)
#     dst_path = os.path.join(dst_dir, filename)
#     shutil.move(src_path, dst_path)

# print("Randomly selected images moved successfully!")

Randomly selected images moved successfully!


In [ ]:
import os
import glob
train_path = '/content/drive/MyDrive/reorganised_data/Train/'
test_path = '/content/drive/MyDrive/reorganised_data/Test/'


# Get the list of class directories
class_dirs = [d for d in os.listdir(train_path) if os.path.isdir(os.path.join(test_path, d))]

# Iterate over the class directories and count the number of images
for class_dir in class_dirs:
    class_path = os.path.join(test_path, class_dir)
    num_images = len(glob.glob(os.path.join(class_path, '*.jpg')))  # Change the file extension to match your dataset
    print(f'{class_dir}: {num_images} images')

bkl: 100 images
nv: 279 images
df: 5 images
mel: 220 images
vasc: 20 images
bcc: 100 images
akiec: 70 images


In [ ]:
from tensorflow.keras.layers import Dropout

In [ ]:
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten

vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in vgg_model.layers[:-2]:
    layer.trainable = False

x = vgg_model.output
x = Flatten()(x)
# x = Dense(512, activation='relu')(x)
# x = Dense(256, activation='relu')(x)
predictions = Dense(7, activation='softmax')(x)

model = Model(inputs=vgg_model.input, outputs=predictions)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

58889256/58889256 [==============================] - 4s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size= 32,
    class_mode='categorical'
)

Found 1770 images belonging to 7 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = train_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size= 32,
    class_mode='categorical'
)

Found 794 images belonging to 7 classes.


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint

filepath = "best_model.h5"


checkpoint = ModelCheckpoint(filepath, monitor='val_acc', save_best_only=True, mode='max', verbose=1)


model_history=model.fit(
  train_generator,
  validation_data=test_generator,
  batch_size= 30,
  epochs=5,callbacks = [checkpoint])

Epoch 1/5
56/56 [==============================] - ETA: 0s - loss: 1.7411 - accuracy: 0.3412 

56/56 [==============================] - 1613s 29s/step - loss: 1.7411 - accuracy: 0.3412 - val_loss: 1.3877 - val_accuracy: 0.4987
Epoch 2/5
56/56 [==============================] - ETA: 0s - loss: 1.3967 - accuracy: 0.4802

56/56 [==============================] - 68s 1s/step - loss: 1.3967 - accuracy: 0.4802 - val_loss: 1.2197 - val_accuracy: 0.5844
Epoch 3/5
56/56 [==============================] - ETA: 0s - loss: 1.2390 - accuracy: 0.5333

56/56 [==============================] - 67s 1s/step - loss: 1.2390 - accuracy: 0.5333 - val_loss: 1.1747 - val_accuracy: 0.5781
Epoch 4/5
56/56 [==============================] - ETA: 0s - loss: 1.1227 - accuracy: 0.5910

56/56 [==============================] - 67s 1s/step - loss: 1.1227 - accuracy: 0.5910 - val_loss: 1.2869 - val_accuracy: 0.5441
Epoch 5/5
56/56 [==============================] - ETA: 0s - loss: 1.0211 - accuracy: 0.6192

56/56 [==============================] - 66s 1s/step - loss: 1.0211 - accuracy: 0.6192 - val_loss: 1.2050 - val_accuracy: 0.5668


In [ ]:
model.save_weights('vgg16_57.h5')

In [ ]:
x = model.predict(test_generator)

In [ ]:
x.shape

In [ ]:
y = np.argmax(x)

In [ ]:
print(y)

In [ ]:
one_hot_preds = np.zeros_like(x)
one_hot_preds[np.arange(len(x)), x.argmax(1)] = 1

# print the one-hot vectors
print(one_hot_preds)

In [ ]:
for row in one_hot_preds:
  print(row)

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Model

input_shape = (224, 224, 3)
IMAGE_SIZE = [224, 224]

base_model1 = MobileNetV2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in base_model1.layers:
    layer.trainable = False

x1 = Flatten()(base_model1.output)



# x = Dense(512, activation='relu')(x)

predictions1 = Dense(7, activation='softmax')(x1)


model1 = Model(inputs=base_model1.input, outputs=predictions1)

model1.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

In [ ]:
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model1_history=model1.fit(
  train_generator,
  validation_data=test_generator,
  batch_size= 30,
  epochs= 5 )

Epoch 1/5
56/56 [==============================] - 76s 1s/step - loss: 2.0911 - accuracy: 0.7209 - val_loss: 4.5089 - val_accuracy: 0.5743
Epoch 2/5
56/56 [==============================] - 70s 1s/step - loss: 1.6760 - accuracy: 0.7740 - val_loss: 4.2773 - val_accuracy: 0.5730
Epoch 3/5
56/56 [==============================] - 64s 1s/step - loss: 1.4807 - accuracy: 0.7797 - val_loss: 4.6420 - val_accuracy: 0.5705
Epoch 4/5
56/56 [==============================] - 68s 1s/step - loss: 1.6774 - accuracy: 0.7740 - val_loss: 4.9117 - val_accuracy: 0.5844
Epoch 5/5
56/56 [==============================] - 66s 1s/step - loss: 1.6271 - accuracy: 0.7864 - val_loss: 6.8887 - val_accuracy: 0.5315


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D

In [ ]:
model3 = Sequential()
model3.add(Conv2D(128, kernel_size = (3,3), input_shape = (224, 224, 3), activation = 'relu', padding = 'same'))
model3.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))
model3.add(MaxPool2D(pool_size = (2,2)))
# model3.add(Dropout(0.3))
model3.add(Conv2D(128, kernel_size = (3,3), activation = 'relu', padding = 'same'))
model3.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))
model3.add(MaxPool2D(pool_size = (2,2)))
# model3.add(Dropout(0.3))
model3.add(Conv2D(128, kernel_size = (3,3), activation = 'relu'))
model3.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))
model3.add(MaxPool2D(pool_size = (2,2)))
# model3.add(Dropout(0.3))
model3.add(Conv2D(128, kernel_size = (3,3), activation = 'relu'))
model3.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))
model3.add(MaxPool2D(pool_size = (2,2)))
# model3.add(Dropout(0.3))
model3.add(Flatten())
model3.add(Dense(128, activation='relu'))
# model3.add(Dropout(0.3))
model3.add(Dense(64, activation='relu'))
model3.add(Dense(32, activation='relu'))
model3.add(Dense(7, activation='softmax'))


In [ ]:
model3.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_50 (Conv2D)          (None, 224, 224, 128)     3584      
                                                                 
 conv2d_51 (Conv2D)          (None, 222, 222, 64)      73792     
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 111, 111, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_52 (Conv2D)          (None, 111, 111, 128)     73856     
                                                                 
 conv2d_53 (Conv2D)          (None, 109, 109, 64)      73792     
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 54, 54, 64)       0         
 g2D)                                                 

In [ ]:
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/hmnist_28_28_RGB.csv')

In [ ]:
data.head()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351,label
0,192,153,193,195,155,192,197,154,185,202,...,173,124,138,183,147,166,185,154,177,2
1,25,14,30,68,48,75,123,93,126,158,...,60,39,55,25,14,28,25,14,27,2
2,192,138,153,200,145,163,201,142,160,206,...,167,129,143,159,124,142,136,104,117,2
3,38,19,30,95,59,72,143,103,119,171,...,44,26,36,25,12,17,25,12,15,2
4,158,113,139,194,144,174,215,162,191,225,...,209,166,185,172,135,149,109,78,92,2


In [ ]:
data.shape

(10015, 2353)

In [ ]:
Y = data['label']

In [ ]:
X = data.drop(columns = 'label')
X.head()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2342,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351
0,192,153,193,195,155,192,197,154,185,202,...,134,173,124,138,183,147,166,185,154,177
1,25,14,30,68,48,75,123,93,126,158,...,82,60,39,55,25,14,28,25,14,27
2,192,138,153,200,145,163,201,142,160,206,...,149,167,129,143,159,124,142,136,104,117
3,38,19,30,95,59,72,143,103,119,171,...,73,44,26,36,25,12,17,25,12,15
4,158,113,139,194,144,174,215,162,191,225,...,201,209,166,185,172,135,149,109,78,92


In [ ]:
X.shape

(10015, 2352)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.2)

In [ ]:
X_test = np.array(X_test).reshape(-1,28,28,3)
X_test = X_test/255

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
oversample = RandomOverSampler()
x,y  = oversample.fit_resample(X_train,Y_train)

In [ ]:
x = np.array(x).reshape(-1,28,28,3)
print('Shape of X :',x.shape)

Shape of X : (37506, 28, 28, 3)


In [ ]:
x.shape

(37506, 28, 28, 3)

In [ ]:
x = x/255

In [ ]:
model = Sequential()
model.add(Conv2D(16, kernel_size = (3,3), input_shape = (28, 28, 3), activation = 'relu', padding = 'same'))
model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))
# model.add(Dropout(0.2))
model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', padding = 'same'))
model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2), padding = 'same'))
# model.add(Dropout(0.3))
model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', padding = 'same'))
model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2), padding = 'same'))
# model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(7, activation='softmax'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_34 (Conv2D)          (None, 28, 28, 16)        448       
                                                                 
 conv2d_35 (Conv2D)          (None, 26, 26, 32)        4640      
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 13, 13, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_36 (Conv2D)          (None, 13, 13, 32)        9248      
                                                                 
 conv2d_37 (Conv2D)          (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 6, 6, 64)         0         
 g2D)                                                 

In [ ]:
callback = tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5',monitor='val acc', mode='max', verbose=1)

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam',metrics = ['accuracy'])


In [ ]:
history = model.fit(x, y, validation_split=0.2,  batch_size = 32, epochs = 5, callbacks=[callback])

Epoch 1/5
937/938 [============================>.] - ETA: 0s - loss: 0.1276 - accuracy: 0.9538
Epoch 1: saving model to best_model.h5
938/938 [==============================] - 9s 7ms/step - loss: 0.1279 - accuracy: 0.9537 - val_loss: 0.9413 - val_accuracy: 0.6517
Epoch 2/5
931/938 [============================>.] - ETA: 0s - loss: 0.1304 - accuracy: 0.9532
Epoch 2: saving model to best_model.h5
938/938 [==============================] - 7s 8ms/step - loss: 0.1299 - accuracy: 0.9534 - val_loss: 0.7185 - val_accuracy: 0.6508
Epoch 3/5
938/938 [==============================] - ETA: 0s - loss: 0.1100 - accuracy: 0.9605
Epoch 3: saving model to best_model.h5
938/938 [==============================] - 6s 7ms/step - loss: 0.1100 - accuracy: 0.9605 - val_loss: 0.4444 - val_accuracy: 0.7887
Epoch 4/5
937/938 [============================>.] - ETA: 0s - loss: 0.1164 - accuracy: 0.9582
Epoch 4: saving model to best_model.h5
938/938 [==============================] - 7s 8ms/step - loss: 0.1163 -

In [ ]:
loss, acc = model.evaluate(X_test, Y_test, verbose=2)

63/63 - 1s - loss: 1.5007 - accuracy: 0.7000 - 739ms/epoch - 12ms/step
